# **Download ultralytics**

In [1]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 61874, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 61874 (delta 198), reused 149 (delta 117), pack-reused 61595 (from 3)
Receiving objects: 100% (61874/61874), 33.59 MiB | 30.15 MiB/s, done.
Resolving deltas: 100% (46031/46031), done.


# install in developer mode

In [2]:
%cd ultralytics
!pip install -e '.[dev]'

/kaggle/working/ultralytics
Obtaining file:///kaggle/working/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 57.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 104.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.3/380.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.3 MB/s eta 0:00:00
  Building editable for ultralytics (pyproject.toml) ... done
  Crea

# check machine configuration and H/W

In [3]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.163 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6407.5/8062.4 GB disk)


# Code to be added to blocks.py

In [ ]:
class ECAConv(nn.Module):
    """Efficient Channel Attention (ECA) module"""
    def __init__(self, c, b=1, gamma=2):
        super().__init__()
        k = int(abs((torch.log2(torch.tensor(c)) + b) / gamma))
        k = k if k % 2 else k + 1  # Ensure k is odd
        self.conv = nn.Conv1d(1, 1, kernel_size=k, padding=k // 2, bias=False)

    def forward(self, x):
        y = torch.mean(x, dim=(2, 3), keepdim=True)  # Global average pooling
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)
        return x * torch.sigmoid(y)

class GSConv(nn.Module):
    """Ghost Shuffle Convolution for efficient feature extraction"""
    def __init__(self, c1, c2, k=3, s=1, p=None, g=1, d=1):
        super().__init__()
        c_ = c2 // 2  # Half channels
        self.primary_conv = nn.Conv2d(c1, c_, k, s, p, groups=g, dilation=d, bias=False)
        self.cheap_conv = nn.Conv2d(c_, c_, 1, 1, bias=False)  # Ghost Feature Generation

    def forward(self, x):
        y1 = self.primary_conv(x)
        y2 = self.cheap_conv(y1)
        return torch.cat([y1, y2], dim=1)  # Channel-wise concatenation

class EfficientConv(nn.Module):
    """Efficient Convolution block combining Conv2d, GSConv, and ECA attention"""
    def __init__(self, c1, c2, k=3, s=1, p=1, g=1, d=1, act=True, use_bn=True, attention=True):
        super().__init__()
        c_half = c2 // 2  # Split channels for Conv2d and GSConv
        self.conv = nn.Conv2d(c1, c_half, k, s, p, groups=g, dilation=d, bias=False)
        self.gsconv = GSConv(c1, c_half, k, s, p, g, d)  # Efficient convolution
        self.bn = nn.BatchNorm2d(c2) if use_bn else nn.Identity()
        self.act = nn.SiLU() if act else nn.Identity()
        self.eca = ECAConv(c2) if attention else nn.Identity()  # Efficient Channel Attention

    def forward(self, x):
        y1 = self.conv(x)
        y2 = self.gsconv(x)
        y = torch.cat([y1, y2], dim=1)  # Concatenation
        return self.act(self.eca(self.bn(y)))  # Final output

In [5]:
%%writefile /kaggle/working/ultralytics/ultralytics/nn/modules/block.py

Overwriting /kaggle/working/ultralytics/ultralytics/nn/modules/block.py


# update tasks.py

In [7]:
%%writefile /kaggle/working/ultralytics/ultralytics/nn/tasks.py

Overwriting /kaggle/working/ultralytics/ultralytics/nn/tasks.py


# update __init__.py

In [9]:
%%writefile /kaggle/working/ultralytics/ultralytics/nn/modules/__init__.py

Overwriting /kaggle/working/ultralytics/ultralytics/nn/modules/__init__.py


## configure updated YOLOv8-model (CFG)

In [11]:
%%writefile /kaggle/working/ultralytics/ultralytics/cfg/models/12/yolov12-our.yaml
nc: 80 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolo12n.yaml' will call yolo12.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.50, 0.25, 1024] # summary: 272 layers, 2,602,288 parameters, 2,602,272 gradients, 6.7 GFLOPs
  s: [0.50, 0.50, 1024] # summary: 272 layers, 9,284,096 parameters, 9,284,080 gradients, 21.7 GFLOPs
  m: [0.50, 1.00, 512] # summary: 292 layers, 20,199,168 parameters, 20,199,152 gradients, 68.1 GFLOPs
  l: [1.00, 1.00, 512] # summary: 488 layers, 26,450,784 parameters, 26,450,768 gradients, 89.7 GFLOPs
  x: [1.00, 1.50, 512] # summary: 488 layers, 59,210,784 parameters, 59,210,768 gradients, 200.3 GFLOPs

# YOLO12n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, EfficientConv, [128, 3, 2]] # 1-P2/4
  - [-1, 2, C3k2, [256, False, 0.25]]
  - [-1, 1, EfficientConv, [256, 3, 2]] # 3-P3/8
  - [-1, 2, C3k2, [512, False, 0.25]]
  - [-1, 1, EfficientConv, [512, 3, 2]] # 5-P4/16
  - [-1, 4, A2C2f, [512, True, 4]]
  - [-1, 1, EfficientConv, [1024, 3, 2]] # 7-P5/32
  - [-1, 4, A2C2f, [1024, True, 1]] # 8

# YOLO12n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 2, A2C2f, [512, False, -1]] # 11

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 2, A2C2f, [256, False, -1]] # 14

  - [-1, 1, EfficientConv, [256, 3, 2]]
  - [[-1, 11], 1, Concat, [1]] # cat head P4
  - [-1, 2, A2C2f, [512, False, -1]] # 17

  - [-1, 1, EfficientConv, [512, 3, 2]]
  - [[-1, 8], 1, Concat, [1]] # cat head P5
  - [-1, 2, C3k2, [1024, True]] # 20 (P5/32-large)

  - [[14, 17, 20], 1, Detect, [nc]] # Detect(P3, P4, P5)


Overwriting /kaggle/working/ultralytics/ultralytics/cfg/models/12/yolov12-our.yaml


# create a new file (lets name it train.py), and check model configuration

In [12]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO

if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/12/yolov12-our.yaml')
    model.info()

Writing train.py


In [13]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
YOLOv12-our summary: 296 layers, 2,441,992 parameters, 2,441,976 gradients, 6.3 GFLOPs


# Everything is file and ready to go

Let's download the dataset from Roboflow

In [14]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="y4uzPDgeB6rEDxdHhxsW")
project = rf.workspace("buddetection").project("bud_detection-np")
version = project.version(1)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 63.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 91.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 105.7 MB/s eta 0:00:0000:01
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.4 requ


Extracting Dataset Version Zip to bud_detection-np-1 in yolov8:: 100%|██████████| 6468/6468 [00:00<00:00, 6895.83it/s]


In [15]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/12/yolov12-our.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/bud_detection-np-1/data.yaml", epochs=100, batch=16, imgsz=640)
    #model.train(resume=True)

Overwriting train.py


# **YOLOv12n + EfficientConv**

In [16]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.163 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/ultralytics/bud_detection-np-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=ultralytics/cfg/models/12/yolov12-our.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, 

# **YOLOV10 + EFFICIENTCONV**

In [19]:
%%writefile /kaggle/working/ultralytics/ultralytics/cfg/models/v10/yolov10n-our.yaml
# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# YOLOv10n object detection model with P3/8 - P5/32 outputs
# Model docs: https://docs.ultralytics.com/models/yolov10
# Task docs: https://docs.ultralytics.com/tasks/detect

# Parameters
nc: 80 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov10n.yaml' will call yolov10.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024]

backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, EfficientConv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, EfficientConv, [256, 3, 2]] # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, SCDown, [512, 3, 2]] # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, SCDown, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]] # 9
  - [-1, 1, PSA, [1024]] # 10

# YOLOv10.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C2f, [512]] # 13

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C2f, [256]] # 16 (P3/8-small)

  - [-1, 1, EfficientConv, [256, 3, 2]]
  - [[-1, 13], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]] # 19 (P4/16-medium)

  - [-1, 1, SCDown, [512, 3, 2]]
  - [[-1, 10], 1, Concat, [1]] # cat head P5
  - [-1, 3, C2fCIB, [1024, True, True]] # 22 (P5/32-large)

  - [[16, 19, 22], 1, v10Detect, [nc]] # Detect(P3, P4, P5)


Writing /kaggle/working/ultralytics/ultralytics/cfg/models/v10/yolov10n-our.yaml


In [20]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO

if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v10/yolov10n-our.yaml')
    model.info()

Overwriting train.py


In [21]:
!python train.py

YOLOv10n-our summary: 235 layers, 2,761,129 parameters, 2,761,113 gradients, 8.6 GFLOPs


In [22]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v10/yolov10n-our.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/bud_detection-np-1/data.yaml", epochs=100, batch=16, imgsz=640)
    

Overwriting train.py


In [23]:
!python train.py

Ultralytics 8.3.163 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/ultralytics/bud_detection-np-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=ultralytics/cfg/models/v10/yolov10n-our.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap

# YOLOV11 + EFFICIENTCONV

In [29]:
%%writefile /kaggle/working/ultralytics/ultralytics/cfg/models/11/yolo11-our.yaml
# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# Ultralytics YOLO11 object detection model with P3/8 - P5/32 outputs
# Model docs: https://docs.ultralytics.com/models/yolo11
# Task docs: https://docs.ultralytics.com/tasks/detect

# Parameters
nc: 80 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolo11n.yaml' will call yolo11.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.50, 0.25, 1024] # summary: 181 layers, 2624080 parameters, 2624064 gradients, 6.6 GFLOPs
  s: [0.50, 0.50, 1024] # summary: 181 layers, 9458752 parameters, 9458736 gradients, 21.7 GFLOPs
  m: [0.50, 1.00, 512] # summary: 231 layers, 20114688 parameters, 20114672 gradients, 68.5 GFLOPs
  l: [1.00, 1.00, 512] # summary: 357 layers, 25372160 parameters, 25372144 gradients, 87.6 GFLOPs
  x: [1.00, 1.50, 512] # summary: 357 layers, 56966176 parameters, 56966160 gradients, 196.0 GFLOPs

# YOLO11n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, EfficientConv, [128, 3, 2]] # 1-P2/4
  - [-1, 2, C3k2, [256, False, 0.25]]
  - [-1, 1, EfficientConv, [256, 3, 2]] # 3-P3/8
  - [-1, 2, C3k2, [512, False, 0.25]]
  - [-1, 1, EfficientConv, [512, 3, 2]] # 5-P4/16
  - [-1, 2, C3k2, [512, True]]
  - [-1, 1, EfficientConv, [1024, 3, 2]] # 7-P5/32
  - [-1, 2, C3k2, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]] # 9
  - [-1, 2, C2PSA, [1024]] # 10

# YOLO11n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 2, C3k2, [512, False]] # 13

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 2, C3k2, [256, False]] # 16 (P3/8-small)

  - [-1, 1, EfficientConv, [256, 3, 2]]
  - [[-1, 13], 1, Concat, [1]] # cat head P4
  - [-1, 2, C3k2, [512, False]] # 19 (P4/16-medium)

  - [-1, 1, EfficientConv, [512, 3, 2]]
  - [[-1, 10], 1, Concat, [1]] # cat head P5
  - [-1, 2, C3k2, [1024, True]] # 22 (P5/32-large)

  - [[16, 19, 22], 1, Detect, [nc]] # Detect(P3, P4, P5)


Overwriting /kaggle/working/ultralytics/ultralytics/cfg/models/11/yolo11-our.yaml


In [32]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO

if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/11/yolo11-our.yaml')
    model.info()

Overwriting train.py


In [33]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
YOLO11-our summary: 205 layers, 2,463,784 parameters, 2,463,768 gradients, 6.2 GFLOPs


In [34]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/11/yolo11-our.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/bud_detection-np-1/data.yaml", epochs=100, batch=16, imgsz=640)
    

Overwriting train.py


In [35]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.163 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/ultralytics/bud_detection-np-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=ultralytics/cfg/models/11/yolo11-our.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, 

# **YOLOv9n + EfficientConv**

In [39]:
%%writefile /kaggle/working/ultralytics/ultralytics/cfg/models/v9/yolov9t-our.yaml
# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# YOLOv9t object detection model with P3/8 - P5/32 outputs
# Model docs: https://docs.ultralytics.com/models/yolov9
# Task docs: https://docs.ultralytics.com/tasks/detect
# 544 layers, 2128720 parameters, 8.5 GFLOPs

# Parameters
nc: 80 # number of classes

# GELAN backbone
backbone:
  - [-1, 1, Conv, [16, 3, 2]] # 0-P1/2
  - [-1, 1, EfficientConv, [32, 3, 2]] # 1-P2/4
  - [-1, 1, ELAN1, [32, 32, 16]] # 2
  - [-1, 1, AConv, [64]] # 3-P3/8
  - [-1, 1, RepNCSPELAN4, [64, 64, 32, 3]] # 4
  - [-1, 1, AConv, [96]] # 5-P4/16
  - [-1, 1, RepNCSPELAN4, [96, 96, 48, 3]] # 6
  - [-1, 1, AConv, [128]] # 7-P5/32
  - [-1, 1, RepNCSPELAN4, [128, 128, 64, 3]] # 8
  - [-1, 1, SPPELAN, [128, 64]] # 9

head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 1, RepNCSPELAN4, [96, 96, 48, 3]] # 12

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 1, RepNCSPELAN4, [64, 64, 32, 3]] # 15

  - [-1, 1, AConv, [48]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 1, RepNCSPELAN4, [96, 96, 48, 3]] # 18 (P4/16-medium)

  - [-1, 1, AConv, [64]]
  - [[-1, 9], 1, Concat, [1]] # cat head P5
  - [-1, 1, RepNCSPELAN4, [128, 128, 64, 3]] # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]] # Detect(P3, P4, P5)


Writing /kaggle/working/ultralytics/ultralytics/cfg/models/v9/yolov9t-our.yaml


In [40]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO

if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v9/yolov9t-our.yaml')
    model.info()

Overwriting train.py


In [41]:
!python train.py

YOLOv9t-our summary: 548 layers, 2,127,635 parameters, 2,127,619 gradients, 8.4 GFLOPs


In [42]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v9/yolov9t-our.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/bud_detection-np-1/data.yaml", epochs=100, batch=16, imgsz=640)
    

Overwriting train.py


In [43]:
!python train.py

Ultralytics 8.3.163 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/ultralytics/bud_detection-np-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=ultralytics/cfg/models/v9/yolov9t-our.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_m

In [45]:
import shutil

# Zip the 'runs' directory
shutil.make_archive("runs_backup_12_11_10_9n_effconv", 'zip', "runs")

print("Zipping complete! File saved as runs_backup_9n_effconv.zip")

Zipping complete! File saved as runs_backup_9n_effconv.zip


# update data.yaml file

In [22]:
%%writefile /kaggle/working/ultralytics/bud_detection-np-1/data.yaml
names:
- BUD
nc: 1
roboflow:
  license: CC BY 4.0
  project: bud_detection-np
  url: https://universe.roboflow.com/buddetection/bud_detection-np/dataset/1
  version: 1
  workspace: buddetection
test: /kaggle/working/ultralytics/bud_detection-np-1/test/images
train: /kaggle/working/ultralytics/bud_detection-np-1/train/images
val: /kaggle/working/ultralytics/bud_detection-np-1/valid/images

Overwriting /kaggle/working/ultralytics/bud_detection-np-1/data.yaml


# update the train.py to enable the training

In [22]:
%%writefile /kaggle/working/ultralytics/ultralytics/cfg/models/12/yolo12-ours.yaml
# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# YOLO12 object detection model with P3/8 - P5/32 outputs
# Model docs: https://docs.ultralytics.com/models/yolo12
# Task docs: https://docs.ultralytics.com/tasks/detect

# Parameters
nc: 80 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolo12n.yaml' will call yolo12.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.50, 0.25, 1024] # summary: 272 layers, 2,602,288 parameters, 2,602,272 gradients, 6.7 GFLOPs
  s: [0.50, 0.50, 1024] # summary: 272 layers, 9,284,096 parameters, 9,284,080 gradients, 21.7 GFLOPs
  m: [0.50, 1.00, 512] # summary: 292 layers, 20,199,168 parameters, 20,199,152 gradients, 68.1 GFLOPs
  l: [1.00, 1.00, 512] # summary: 488 layers, 26,450,784 parameters, 26,450,768 gradients, 89.7 GFLOPs
  x: [1.00, 1.50, 512] # summary: 488 layers, 59,210,784 parameters, 59,210,768 gradients, 200.3 GFLOPs

# YOLO12n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, EfficientConv, [128, 3, 2]] # 1-P2/4
  - [-1, 2, C3k2, [256, False, 0.25]]
  - [-1, 1, EfficientConv, [256, 3, 2]] # 3-P3/8
  - [-1, 2, C3k2, [512, False, 0.25]]
  - [-1, 1, EfficientConv, [512, 3, 2]] # 5-P4/16
  - [-1, 4, A2C2f, [512, True, 4]]
  - [-1, 1, EfficientConv, [1024, 3, 2]] # 7-P5/32
  - [-1, 4, A2C2f, [1024, True, 1]] # 8

# YOLO12n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 2, A2C2f, [512, False, -1]] # 11

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 2, A2C2f, [256, False, -1]] # 14

  - [-1, 1, EfficientConv, [256, 3, 2]]
  - [[-1, 11], 1, Concat, [1]] # cat head P4
  - [-1, 2, A2C2f, [512, False, -1]] # 17

  - [-1, 1, EfficientConv, [512, 3, 2]]
  - [[-1, 8], 1, Concat, [1]] # cat head P5
  - [-1, 2, C3k2, [1024, True]] # 20 (P5/32-large)

  - [[14, 17, 20], 1, Detect, [nc]] # Detect(P3, P4, P5)


Writing /kaggle/working/ultralytics/ultralytics/cfg/models/12/yolo12-ours.yaml


In [26]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO

if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/12/yolo12-ours.yaml')
    model.info()

Overwriting train.py


In [27]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
YOLO12-ours summary: 288 layers, 2,486,784 parameters, 2,486,768 gradients, 6.3 GFLOPs


# !!!START THE TRAINING!!!

# ** YOLOv8n-our**

In [21]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.90 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-mobile.yaml, data=/kaggle/working/ultralytics/bud_detection-np-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=20, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fa

# RESULTS ARE HERE!!!

Download the results

In [22]:
!zip -r file.zip runs

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/results.csv (deflated 61%)
  adding: runs/detect/train/val_batch1_pred.jpg (deflated 5%)
  adding: runs/detect/train/val_batch1_labels.jpg (deflated 5%)
  adding: runs/detect/train/labels.jpg (deflated 24%)
  adding: runs/detect/train/train_batch1.jpg (deflated 1%)
  adding: runs/detect/train/labels_correlogram.jpg (deflated 35%)
  adding: runs/detect/train/args.yaml (deflated 53%)
  adding: runs/detect/train/train_batch14801.jpg (deflated 5%)
  adding: runs/detect/train/val_batch2_pred.jpg (deflated 6%)
  adding: runs/detect/train/events.out.tfevents.1742012830.e25b78843490.119.0 (deflated 88%)
  adding: runs/detect/train/P_curve.png (deflated 17%)
  adding: runs/detect/train/val_batch0_labels.jpg (deflated 5%)
  adding: runs/detect/train/train_batch0.jpg (deflated 1%)
  adding: runs/detect/train/results.png (deflated 9%)
  adding: runs/detect/train/train

# Below YOLOv10n results are collected on sugarcane bud detection dataset

In [23]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v10/yolov10n.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/bud_detection-np-1/data.yaml", epochs=100, batch=32, imgsz=640)
    #model.train(resume=True)

Overwriting train.py


In [24]:
!python train.py

Ultralytics 8.3.90 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v10/yolov10n.yaml, data=/kaggle/working/ultralytics/bud_detection-np-1/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

# This is a proposed model training on VisDrone Dataset 

In [13]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v8/yolov8-mobile.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="VisDrone.yaml", epochs=50, batch=32, imgsz=640)
    #model.train(resume=True)

Overwriting train.py


In [14]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.90 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-mobile.yaml, data=VisDrone.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sav